In [2]:
%load_ext sql
%sql sqlite:///northwind.db

In [5]:
%%sql
ALTER TABLE "Order Details" RENAME COLUMN UnitPrice TO Price;
ALTER TABLE Products RENAME COLUMN UnitPrice TO Price;


 * sqlite:///northwind.db
Done.
Done.


[]